# DECODE - Fit SMLM Data
The purpose of this notebook is to demonstrate the fitting procedure for a pretrained model.
Please be advised to have a read of the Introduction notebook first.

In [ ]:
# To be removed
import os
os.chdir('/home/lucas/RemoteDeploy/DeepSMLM/')

In [ ]:
import decode
import decode.utils

import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

## Set parameters
Set device for inference (i.e. CUDA vs. CPU) and some other parameters which only need to be changed if you know what
 you are doing.

In [2]:
device = 'cuda'  # or 'cpu'

# change only if you know what you are doing
post_raw_th = 0.1  # post-processing initial threshold

## Load Example data
We provide several example data for trying out DECODE.
For this we load a gateway file which includes the links to the respective data.

In [ ]:
gateway = decode.utils.example_helper.load_gateway()

# dir where to store example data, leave as '' to store in current folder
path = Path('')

# change here for other files
package = gateway['examples']['package_1']
frame_path, model_path, param_path = decode.utils.example_helper.load_example_package(path=path / package['name'],
                                                                        url=package['url'], hash=package['hash'])

## Load Frames
Load TIFF file.
Change only needed if you load a custom tif file.
Note that the TIFF loader will auto-load and concatenate tiff files
that are consecutive and share
the same meta data. For example specifying `dummy.tif` will also load  `dummy_0.tif, dummy_1.tif` if present in the
same folder.
If you have single page tiff files, you can also specify a folder.

In [3]:
# depends on your input, e.g. load a tiff
frames = decode.utils.frames_io.load_tif(frame_path)

## Load Parameters and Model
Specify Post-Processing as by the parameter file you trained the model with

In [4]:
param = decode.utils.param_io.load_params(param_path)
model = decode.neuralfitter.models.SigmaMUNet.parse(param)
model = decode.utils.model_io.LoadSaveModel(model,
                                              input_file=model_path,
                                              output_file=None).load_init()

Model instantiated.
Model SHA-1 hash: 3dc4813290eac0af7b8653ef1dc37f12c9de73ba
Loaded pretrained model: /home/lucas/git/thesisjupyterbooks/temp/Artur_Bundle/model_out_0.pt


## Set Up Pre- and Post-Processing

In [1]:
# determine extent of frame and its dimension
frame_extent = ((-0.5, frames.size(-2) - 0.5), (-0.5, frames.size(-1) - 0.5))
frame_size = tuple(frames.size()[-2:])

# setup frame processing as by the parameter with which the model was trained
frame_proc = decode.neuralfitter.scale_transform.AmplitudeRescale.parse(param)

# Setup post-processing
# It's a sequence of backscaling, relative to abs. coord conversion and frame2emitter conversion
post_proc = decode.neuralfitter.utils.processing.TransformSequence([
    
    decode.neuralfitter.scale_transform.InverseParamListRescale.parse(param),
    
    decode.neuralfitter.coord_transform.Offset2Coordinate(xextent=frame_extent[0],
                                                            yextent=frame_extent[1],
                                                            img_shape=frame_size),
    
    decode.neuralfitter.post_processing.LookUpPostProcessing(raw_th=post_raw_th,
                                                               xy_unit='px', 
                                                               px_size=param.Camera.px_size)
    
    
])

NameError: name 'frames' is not defined

## Fit the Data

In [6]:
infer = decode.neuralfitter.Infer(model=model, ch_in=param.HyperParameter.channels_in,
                                    frame_proc=frame_proc, post_proc=post_proc,
                                    device=device)

emitter = infer.forward(frames)

100%|██████████| 16/16 [00:01<00:00,  9.50it/s]


In [7]:
# check on the output
print(emitter)

EmitterSet
::num emitters: 158000
::xy unit: px
::frame range: 0 - 999
::spanned volume: [   0.20551562    0.19100158 -105.857735  ] - [ 62.863    63.06127 119.09278]


## Export EmitterSet
e.g. as CSV, or as pickle object for reload in another notebook etc.

In [8]:
# csv export
decode.utils.emitter_io.save_csv('emitter.csv', emitter.to_dict())

# pickle
emitter.save('emitter.pl')  # can be loaded via 'decode.EmitterSet.load('emitter.pl')'
